# ___Imports___

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
import plotly.express as px
from geopy.geocoders import Nominatim
import geopy as gp
from datetime import datetime
from glob import glob

## ___NASA data___

In [14]:
nasa_data = pd.read_csv('../data/meteorite-landings.csv')
print(nasa_data.columns)

nasa_data.head()
np.shape(nasa_data)

Index(['name', 'id', 'nametype', 'recclass', 'mass', 'fall', 'year', 'reclat',
       'reclong', 'GeoLocation'],
      dtype='object')


(45716, 10)

## ___IMO data___

In [12]:
imo_data = pd.read_csv('../data/IMO/data/IMO9920.csv')
print(imo_data.columns)

imo_data.head()
np.shape(imo_data)

Index(['Session ID', 'Start Date', 'Observer ID', 'Submitter ID',
       'Actual Observer Name', 'Submitted by', 'City', 'Country', 'Latitude',
       'Longitude', 'Elevation'],
      dtype='object')


(42989, 11)

## ___Combining data___

In [22]:
all_data = pd.concat([nasa_data, imo_data], sort=False, keys=['NASA', 'IMO'])
print(all_data.columns)
print(all_data.head)
np.shape(all_data)


Index(['name', 'id', 'nametype', 'recclass', 'mass', 'fall', 'year', 'reclat',
       'reclong', 'GeoLocation', 'Session ID', 'Start Date', 'Observer ID',
       'Submitter ID', 'Actual Observer Name', 'Submitted by', 'City',
       'Country', 'Latitude', 'Longitude', 'Elevation'],
      dtype='object')
<bound method NDFrame.head of                 name     id nametype     recclass      mass  fall    year  \
NASA 0        Aachen    1.0    Valid           L5      21.0  Fell  1880.0   
     1        Aarhus    2.0    Valid           H6     720.0  Fell  1951.0   
     2          Abee    6.0    Valid          EH4  107000.0  Fell  1952.0   
     3      Acapulco   10.0    Valid  Acapulcoite    1914.0  Fell  1976.0   
     4       Achiras  370.0    Valid           L6     780.0  Fell  1902.0   
...              ...    ...      ...          ...       ...   ...     ...   
IMO  42984       NaN    NaN      NaN          NaN       NaN   NaN     NaN   
     42985       NaN    NaN      NaN          NaN

(88705, 21)

## ___Cleaning___